# Grad-CAM Refinement & Testing (Week 2, Day 4)

## Notebook Purpose

This notebook (`04_gradcam_refinement.ipynb`) extends the PneumoDetect explainability workflow by refining and testing the Grad-CAM implementation introduced in Week 2 Day 3.

### Objectives
- Encapsulate Grad-CAM inference in a reusable `generate_cam(image_path, model_path)` function.
- Validate reliability and reproducibility with unit and integration tests.
- Enhance color rendering using OpenCV’s `COLORMAP_JET`.
- Ensure Grad-CAM overlays render consistently across different checkpoints.
- Summarize and visualize Grad-CAM results for misclassified examples.

### Technical Summary
- **Module Updated:** `src/gradcam.py`
- **Tests Added:** `tests/test_gradcam_refinement.py`, `tests/test_gradcam_refinement_integration.py`
- **Outputs Saved To:** `reports/week2_gradcam_refinement/`

### Deliverables
- Passing test suite (`pytest`): verifies shape, normalization (0–1), and overlay integrity.
- Reliable Grad-CAM visualizations confirming robust explainability.

In [2]:
# Add script to include project root in sys.path

import sys
from pathlib import Path

# Add project root (one level up from notebooks/)
project_root = Path(__file__).resolve().parent.parent if "__file__" in globals() else Path.cwd().parent
sys.path.append(str(project_root))

In [3]:
# Import necessary libraries

from pathlib import Path
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from src.gradcam import GradCAM, generate_cam

In [4]:
# ------------------------------------------------------------
# 1. Setup & Directories
# ------------------------------------------------------------
project_root = Path.cwd().parents[0] if Path.cwd().name == 'notebooks' else Path.cwd()
output_dir = project_root / 'reports' / 'week2_gradcam_refinement'
output_dir.mkdir(parents=True, exist_ok=True)

model_path = project_root / 'saved_models' / 'resnet50_baseline.pt'
finetuned_model_path = project_root / 'saved_models' / 'resnet50_finetuned.pt'
image_dir = project_root / 'data' / 'rsna_subset' / 'train_images'

print(f"Using models from: {model_path.parent}")
print(f"Saving refined outputs to: {output_dir}")

Using models from: /Users/adrianadewunmi/VSCODE/AI-Assisted-Pneumonia-Detection-Project/saved_models
Saving refined outputs to: /Users/adrianadewunmi/VSCODE/AI-Assisted-Pneumonia-Detection-Project/reports/week2_gradcam_refinement
